%md
WARSTWA BRĄZOWA - ARCHITEKTURA MEDALIONU

Autor: Weronika Wąsikowska

Opis: Pełna implementacja warstwy brązowej dla danych Stack Exchange

In [0]:
from pyspark.sql import SparkSession

# Tworzenie sesji Spark
spark = SparkSession.builder.appName("BronzeLayer").getOrCreate()

In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/Badges.xml,Badges.xml,18158773,1741787269000
dbfs:/FileStore/tables/Comments.xml,Comments.xml,35074960,1741787275000
dbfs:/FileStore/tables/PostHistory.xml,PostHistory.xml,207523334,1741787345000
dbfs:/FileStore/tables/PostLinks.xml,PostLinks.xml,622527,1741787276000
dbfs:/FileStore/tables/Posts.xml,Posts.xml,94459301,1741787313000
dbfs:/FileStore/tables/Tags.xml,Tags.xml,345262,1741787314000
dbfs:/FileStore/tables/Users.xml,Users.xml,26064495,1741787324000
dbfs:/FileStore/tables/Votes.xml,Votes.xml,63388433,1741787346000


In [0]:
# Ścieżki do plików XML
file_path_badges = "/FileStore/tables/Badges.xml"
file_path_posts = "/FileStore/tables/Posts.xml"
file_path_users = "/FileStore/tables/Users.xml"
file_path_comments = "/FileStore/tables/Comments.xml"
file_path_tags = "/FileStore/tables/Tags.xml"
file_path_votes = "/FileStore/tables/Votes.xml"
file_path_post_links = "/FileStore/tables/PostLinks.xml"
file_path_post_history = "/FileStore/tables/PostHistory.xml"


In [0]:
# Ładowanie danych z pliku XML do DataFrame’ów
df_badges = spark.read.format("xml").option("rowTag","row").load(file_path_badges)
df_posts = spark.read.format("xml").option("rowTag","row").load(file_path_posts)
df_users = spark.read.format("xml").option("rowTag","row").load(file_path_users)
df_comments = spark.read.format("xml").option("rowTag","row").load(file_path_comments)
df_tags = spark.read.format("xml").option("rowTag","row").load(file_path_tags)
df_votes = spark.read.format("xml").option("rowTag","row").load(file_path_votes)
df_post_links = spark.read.format("xml").option("rowTag","row").load(file_path_post_links)
df_post_history = spark.read.format("xml").option("rowTag","row").load(file_path_post_history)


In [0]:
# Sprawdzenie czy dane zostały poprawnie wczytane 
df_badges.show(5)
df_posts.show(5)
df_users.show(5)
df_comments.show(5)
df_tags.show(5)
df_votes.show(5)
df_post_links.show(5)
df_post_history.show(5)

+------+--------------------+---+--------------+---------+-------+
|_Class|               _Date|_Id|         _Name|_TagBased|_UserId|
+------+--------------------+---+--------------+---------+-------+
|     3|2011-11-30 18:10:...|  1|Autobiographer|    false|      1|
|     3|2011-11-30 18:15:...|  2|Autobiographer|    false|      2|
|     3|2011-11-30 18:15:...|  3|Autobiographer|    false|      3|
|     3|2011-11-30 18:45:...|  4|Autobiographer|    false|      5|
|     3|2011-11-30 18:45:...|  5|Autobiographer|    false|      7|
+------+--------------------+---+--------------+---------+-------+
only showing top 5 rows

+-----------------+------------+--------------------+-----------+-------------+-------------------+---------------+--------------------+--------------+---+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_

Analiza danych

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import countDistinct

# Wyświetlenie schematów tabel
print("Struktura tabel w warstwie brązowej:")
for name, df in {
    "badges": df_badges,
    "posts": df_posts,
    "users": df_users,
    "comments": df_comments,
    "tags": df_tags,
    "votes": df_votes,
    "post_links": df_post_links,
    "post_history": df_post_history
}.items():
    print(f"\n Schemat tabeli: {name}")
    df.printSchema()

Struktura tabel w warstwie brązowej:

 Schemat tabeli: badges
root
 |-- _Class: long (nullable = true)
 |-- _Date: timestamp (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _Name: string (nullable = true)
 |-- _TagBased: boolean (nullable = true)
 |-- _UserId: long (nullable = true)


 Schemat tabeli: posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _ContentLicense: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (null

In [0]:
# Sprawdzenie liczby rekordów w tabelach
print("\n Liczba rekordów w tabelach:")
for name, df in {
    "badges": df_badges,
    "posts": df_posts,
    "users": df_users,
    "comments": df_comments,
    "tags": df_tags,
    "votes": df_votes,
    "post_links": df_post_links,
    "post_history": df_post_history
}.items():
    print(f"{name}: {df.count()} rekordów")


 Liczba rekordów w tabelach:
badges: 159496 rekordów
posts: 64988 rekordów
users: 75729 rekordów
comments: 103194 rekordów
tags: 4774 rekordów
votes: 688874 rekordów
post_links: 5439 rekordów
post_history: 246174 rekordów


In [0]:
from pyspark.sql.functions import countDistinct, max

print("\n Sprawdzanie unikalności i poprawności zakresu _Id w tabelach:")

for name, df in {
    "badges": df_badges,
    "posts": df_posts,
    "users": df_users,
    "comments": df_comments,
    "tags": df_tags,
    "votes": df_votes,
    "post_links": df_post_links,
    "post_history": df_post_history
}.items():
    if "_Id" in df.columns:
        total_count = df.count()
        unique_count = df.select(countDistinct("_Id")).collect()[0][0]
        max_id = df.select(max("_Id")).collect()[0][0]
        
        print(f"\n {name}:")
        print(f"   - {unique_count} unikalnych wartości _Id na {total_count} rekordów")
        print(f"   - Maksymalna wartość _Id: {max_id}")



 Sprawdzanie unikalności i poprawności zakresu _Id w tabelach:

 badges:
   - 159496 unikalnych wartości _Id na 159496 rekordów
   - Maksymalna wartość _Id: 188551

 posts:
   - 64988 unikalnych wartości _Id na 64988 rekordów
   - Maksymalna wartość _Id: 122259

 users:
   - 75729 unikalnych wartości _Id na 75729 rekordów
   - Maksymalna wartość _Id: 103794

 comments:
   - 103194 unikalnych wartości _Id na 103194 rekordów
   - Maksymalna wartość _Id: 217512

 tags:
   - 4774 unikalnych wartości _Id na 4774 rekordów
   - Maksymalna wartość _Id: 9391

 votes:
   - 688874 unikalnych wartości _Id na 688874 rekordów
   - Maksymalna wartość _Id: 794163

 post_links:
   - 5439 unikalnych wartości _Id na 5439 rekordów
   - Maksymalna wartość _Id: 2281399

 post_history:
   - 246174 unikalnych wartości _Id na 246174 rekordów
   - Maksymalna wartość _Id: 482277


In [0]:
from pyspark.sql.functions import count

print("\n Sprawdzanie duplikatów na podstawie wszystkich kolumn poza _Id:")

df_list = {
    "badges": df_badges,
    "posts": df_posts,
    "comments": df_comments,
    "votes": df_votes,
    "post_history": df_post_history,
    "post_links": df_post_links,
    "users": df_users,
    "tags": df_tags
}

# Automatyczne wykrywanie kolumn poza _Id
duplicate_checks = {table: [col for col in df.columns if col != "_Id"] for table, df in df_list.items()}

for table, columns in duplicate_checks.items():
    if table in df_list:
        print(f"\n {table}:")
        total_count = df_list[table].count()
        unique_count = df_list[table].dropDuplicates(columns).count()
        duplicates = total_count - unique_count
        print(f"liczba potencjalnych duplikatów: {duplicates}")



 Sprawdzanie duplikatów na podstawie wszystkich kolumn poza _Id:

 badges:
liczba potencjalnych duplikatów: 224

 posts:
liczba potencjalnych duplikatów: 0

 comments:
liczba potencjalnych duplikatów: 0

 votes:
liczba potencjalnych duplikatów: 265542

 post_history:
liczba potencjalnych duplikatów: 0

 post_links:
liczba potencjalnych duplikatów: 0

 users:
liczba potencjalnych duplikatów: 0

 tags:
liczba potencjalnych duplikatów: 0


In [0]:
from pyspark.sql import functions as F

# Tabele, w których wykryto duplikaty
tables_with_duplicates = ["badges", "votes"]

print("\n Wyświetlanie duplikatów tylko w tabelach, w których je wykryto:")

for table in tables_with_duplicates:
    if table in df_list:
        # Pobranie wszystkich kolumn poza `_Id`
        columns = [col for col in df_list[table].columns if col != "_Id"]

        if columns:
            print(f"\n {table}:")
            
            # Grupowanie i filtrowanie duplikatów
            dupe_df = df_list[table].groupBy(columns).count().filter(F.col("count") > 1)
            
            if dupe_df.count() > 0:
                dupe_df.show(truncate=False, n=100)
            else:
                print(f"Brak duplikatów w tabeli {table}.")





 Wyświetlanie duplikatów tylko w tabelach, w których je wykryto:

 badges:
+------+-----------------------+----------------+---------+-------+-----+
|_Class|_Date                  |_Name           |_TagBased|_UserId|count|
+------+-----------------------+----------------+---------+-------+-----+
|2     |2016-10-24 18:29:18.34 |Enlightened     |false    |4386   |2    |
|3     |2019-01-10 19:10:13.8  |Nice Question   |false    |40263  |2    |
|2     |2019-06-18 17:40:13.563|Lifejacket      |false    |23541  |2    |
|3     |2016-10-23 01:54:01.47 |Custodian       |false    |41249  |2    |
|3     |2018-01-01 15:26:54.08 |Nice Answer     |false    |4386   |2    |
|2     |2019-06-18 17:40:13.563|Lifejacket      |false    |5315   |2    |
|3     |2015-12-18 21:47:58.99 |Custodian       |false    |6234   |2    |
|3     |2018-08-14 20:35:19.003|Custodian       |false    |21877  |2    |
|3     |2012-09-21 23:20:17.88 |Custodian       |false    |52     |4    |
|2     |2017-08-11 20:03:42.987|Guru

In [0]:
from pyspark.sql import functions as F

# Analiza brakujących wartości
print("\n Analiza brakujących wartości w tabelach:")

for name, df in df_list.items():
    print(f"\n Tabela {name} - brakujące wartości:")
    df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns
    ]).show()



 Analiza brakujących wartości w tabelach:

 Tabela badges - brakujące wartości:
+------+-----+---+-----+---------+-------+
|_Class|_Date|_Id|_Name|_TagBased|_UserId|
+------+-----+---+-----+---------+-------+
|     0|    0|  0|    0|        0|      0|
+------+-----+---+-----+---------+-------+


 Tabela posts - brakujące wartości:
+-----------------+------------+-----+-----------+-------------+-------------------+---------------+-------------+--------------+---+-----------------+-------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|_AcceptedAnswerId|_AnswerCount|_Body|_ClosedDate|_CommentCount|_CommunityOwnedDate|_ContentLicense|_CreationDate|_FavoriteCount|_Id|_LastActivityDate|_LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|_Tags|_Title|_ViewCount|
+-----------------+------------+-----+-----------+-------------+-------------------+

In [0]:
#Sprawdzenie poprawności dat

date_columns = {
    "posts": "_CreationDate",
    "comments": "_CreationDate",
    "votes": "_CreationDate",
    "users": "_CreationDate"
}

print("\n Analiza zakresu dat:")
for table, column in date_columns.items():
    if table in df_list and column in df_list[table].columns:
        print(f"\n {table} - najwcześniejsza i najpóźniejsza data w {column}:")
        df_list[table].select(F.min(column).alias("Earliest"), F.max(column).alias("Latest")).show()



 Analiza zakresu dat:

 posts - najwcześniejsza i najpóźniejsza data w _CreationDate:
+--------------------+--------------------+
|            Earliest|              Latest|
+--------------------+--------------------+
|2011-10-03 15:29:...|2024-03-01 12:36:...|
+--------------------+--------------------+


 comments - najwcześniejsza i najpóźniejsza data w _CreationDate:
+--------------------+--------------------+
|            Earliest|              Latest|
+--------------------+--------------------+
|2011-10-05 03:53:...|2024-03-01 16:27:...|
+--------------------+--------------------+


 votes - najwcześniejsza i najpóźniejsza data w _CreationDate:
+-------------------+-------------------+
|           Earliest|             Latest|
+-------------------+-------------------+
|2011-10-03 00:00:00|2024-03-01 00:00:00|
+-------------------+-------------------+


 users - najwcześniejsza i najpóźniejsza data w _CreationDate:
+--------------------+--------------------+
|            Earliest

In [0]:
# Sprawdzenie rozkładu danych

from pyspark.sql import functions as F

print("\n Analiza skrajnych wartości w kolumnach liczbowych:")

for table, df in df_list.items():
    numeric_columns = [c for c, t in df.dtypes if t in ["int", "bigint", "double", "float", "long"]]
    
    if numeric_columns:
        print(f"\n🔹 {table} - statystyki dla kolumn liczbowych:")
        stats = [F.min(c).alias(f"min_{c}") for c in numeric_columns] + [F.max(c).alias(f"max_{c}") for c in numeric_columns]
        df.select(stats).show()
    else:
        print(f"\n {table} - brak kolumn liczbowych do analizy.")





 Analiza skrajnych wartości w kolumnach liczbowych:

🔹 badges - statystyki dla kolumn liczbowych:
+----------+-------+-----------+----------+-------+-----------+
|min__Class|min__Id|min__UserId|max__Class|max__Id|max__UserId|
+----------+-------+-----------+----------+-------+-----------+
|         1|      1|          1|         3| 188551|     103791|
+----------+-------+-----------+----------+-------+-----------+


🔹 posts - statystyki dla kolumn liczbowych:
+---------------------+----------------+-----------------+------------------+-------+---------------------+----------------+-------------+---------------+----------+--------------+---------------------+----------------+-----------------+------------------+-------+---------------------+----------------+-------------+---------------+----------+--------------+
|min__AcceptedAnswerId|min__AnswerCount|min__CommentCount|min__FavoriteCount|min__Id|min__LastEditorUserId|min__OwnerUserId|min__ParentId|min__PostTypeId|min__Score|min__ViewC

Raport i pobieranie

In [0]:
import pandas as pd

# Lista do przechowywania wyników
report_data = []

#  1. Analiza duplikatów
for table, df in df_list.items():
    # Pobranie wszystkich kolumn poza `_Id`
    columns = [col for col in df.columns if col != "_Id"]
    
    if columns:
        dupe_count = df.groupBy(columns).count().filter(F.col("count") > 1).count()
        report_data.append({"Tabela": table, "Typ analizy": "Duplikaty", "Wartość": dupe_count})

#  2. Analiza brakujących wartości
for table, df in df_list.items():
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    
    for col, count in null_counts.items():
        if count > 0:  # Tylko kolumny, gdzie są braki
            report_data.append({"Tabela": table, "Typ analizy": f"Braki ({col})", "Wartość": count})

#  3. Analiza wartości skrajnych
for table, df in df_list.items():
    numeric_columns = [c for c, t in df.dtypes if t in ["int", "bigint", "double", "float", "long"]]
    
    if numeric_columns:
        min_max_values = df.select(
            [F.min(c).alias(f"min_{c}") for c in numeric_columns] + 
            [F.max(c).alias(f"max_{c}") for c in numeric_columns]
        ).collect()[0].asDict()
        
        for key, value in min_max_values.items():
            report_data.append({"Tabela": table, "Typ analizy": key, "Wartość": value})

# Konwersja listy do DataFrame Pandas
report_df = pd.DataFrame(report_data)

# **Zapis pliku CSV do lokalizacji dostępnej w Databricks**
csv_path = "/tmp/analiza_danych.csv"
report_df.to_csv(csv_path, index=False)

print(f" Raport został zapisany w {csv_path}. Możesz pobrać go z Databricks.")


 Raport został zapisany w /tmp/analiza_danych.csv. Możesz pobrać go z Databricks.


In [0]:
dbutils.fs.cp("file:/tmp/analiza_danych.csv", "dbfs:/FileStore/analiza_danych.csv")

Out[15]: True

In [0]:
display(dbutils.fs.ls("/FileStore/"))

path,name,size,modificationTime
dbfs:/FileStore/analiza_danych.csv,analiza_danych.csv,3249,1743712281000
dbfs:/FileStore/bronze/,bronze/,0,0
dbfs:/FileStore/export/,export/,0,0
dbfs:/FileStore/export-ready/,export-ready/,0,0
dbfs:/FileStore/gold/,gold/,0,0
dbfs:/FileStore/silver/,silver/,0,0
dbfs:/FileStore/tables/,tables/,0,0


In [0]:
displayHTML(f'<a href="/files/analiza_danych.csv">Kliknij tutaj, aby pobrać plik</a>')

Kliknij tutaj, aby pobrać plik

In [0]:
# Ścieżki docelowe do przechowywania danych w formacie Delta
bronze_path = "dbfs:/FileStore/bronze/"

delta_path_badges = bronze_path + "badges"
delta_path_comments = bronze_path + "comments"
delta_path_post_history = bronze_path + "post_history"
delta_path_post_links = bronze_path + "post_links"
delta_path_posts = bronze_path + "posts"
delta_path_tags = bronze_path + "tags"
delta_path_users = bronze_path + "users"
delta_path_votes = bronze_path + "votes"


In [0]:
# Zapisywanie DataFrame’ów do formatu Delta
df_badges.write.format("delta").mode("overwrite").save(delta_path_badges)
df_comments.write.format("delta").mode("overwrite").save(delta_path_comments)
df_post_history.write.format("delta").mode("overwrite").save(delta_path_post_history)
df_post_links.write.format("delta").mode("overwrite").save(delta_path_post_links)
df_posts.write.format("delta").mode("overwrite").save(delta_path_posts)
df_tags.write.format("delta").mode("overwrite").save(delta_path_tags)
df_users.write.format("delta").mode("overwrite").save(delta_path_users)
df_votes.write.format("delta").mode("overwrite").save(delta_path_votes)

display(dbutils.fs.ls("/FileStore/bronze/"))

print("Dane zostały zapisane w formacie Delta Lake w warstwie brązowej.")

path,name,size,modificationTime
dbfs:/FileStore/bronze/badges/,badges/,0,0
dbfs:/FileStore/bronze/comments/,comments/,0,0
dbfs:/FileStore/bronze/post_history/,post_history/,0,0
dbfs:/FileStore/bronze/post_links/,post_links/,0,0
dbfs:/FileStore/bronze/posts/,posts/,0,0
dbfs:/FileStore/bronze/tags/,tags/,0,0
dbfs:/FileStore/bronze/users/,users/,0,0
dbfs:/FileStore/bronze/votes/,votes/,0,0


Dane zostały zapisane w formacie Delta Lake w warstwie brązowej.


In [0]:
for table_name in ["badges", "comments", "post_history", "post_links", "posts", "tags", "users", "votes"]:
    df = spark.read.format("delta").load(f"dbfs:/FileStore/bronze/{table_name}")
    print(f"Tabela: {table_name}, liczba rekordów: {df.count()}")


Tabela: badges, liczba rekordów: 159496
Tabela: comments, liczba rekordów: 103194
Tabela: post_history, liczba rekordów: 246174
Tabela: post_links, liczba rekordów: 5439
Tabela: posts, liczba rekordów: 64988
Tabela: tags, liczba rekordów: 4774
Tabela: users, liczba rekordów: 75729
Tabela: votes, liczba rekordów: 688874


In [0]:
dbutils.fs.refreshMounts()


Mounts successfully refreshed.
Out[21]: True

In [0]:
display(dbutils.fs.ls("/FileStore/bronze/"))

path,name,size,modificationTime
dbfs:/FileStore/bronze/badges/,badges/,0,0
dbfs:/FileStore/bronze/comments/,comments/,0,0
dbfs:/FileStore/bronze/post_history/,post_history/,0,0
dbfs:/FileStore/bronze/post_links/,post_links/,0,0
dbfs:/FileStore/bronze/posts/,posts/,0,0
dbfs:/FileStore/bronze/tags/,tags/,0,0
dbfs:/FileStore/bronze/users/,users/,0,0
dbfs:/FileStore/bronze/votes/,votes/,0,0


In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/bronze/badges/"))


path,name,size,modificationTime
dbfs:/FileStore/bronze/badges/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/bronze/badges/part-00000-06ba7b25-bb0c-4021-a508-173d1cca047c-c000.snappy.parquet,part-00000-06ba7b25-bb0c-4021-a508-173d1cca047c-c000.snappy.parquet,2667549,1741789924000
dbfs:/FileStore/bronze/badges/part-00000-1af0d772-df75-4b08-b659-c725e1676251-c000.snappy.parquet,part-00000-1af0d772-df75-4b08-b659-c725e1676251-c000.snappy.parquet,2667549,1743712293000
dbfs:/FileStore/bronze/badges/part-00000-53faad06-2ebe-494d-a042-5de9730ef12b-c000.snappy.parquet,part-00000-53faad06-2ebe-494d-a042-5de9730ef12b-c000.snappy.parquet,2667549,1742395655000
dbfs:/FileStore/bronze/badges/part-00000-82abec67-0d59-41aa-ae16-fcb5080d0060-c000.snappy.parquet,part-00000-82abec67-0d59-41aa-ae16-fcb5080d0060-c000.snappy.parquet,2667549,1741791136000


In [0]:
df_badges = spark.read.format("delta").load("dbfs:/FileStore/bronze/badges/")
print(f"Rozmiar tabeli badges (w pamięci Spark): {df_badges.storageLevel}")


Rozmiar tabeli badges (w pamięci Spark): Serialized 1x Replicated
